# Create data

In [ ]:
import json
from pycocotools.coco import COCO
from collections import Counter

In [ ]:
jsonfiles = ["/root/data/small_pen_data_collection/body_parts_detection_merged.json"]

In [ ]:
eye_center_data = []
for jfile in jsonfiles:
    coco = COCO(jfile)
    image_ids = coco.getImgIds()
    cats = coco.cats

    for image_id in image_ids:
        image_data = coco.loadImgs([image_id])[0]
        if "local_path" not in image_data:
            continue

        annotation_ids = coco.getAnnIds(imgIds=[image_data['id']] )   
        headid = coco.getAnnIds(imgIds=[image_data['id']], catIds=[2])
        if len(coco.loadAnns(headid)) > 0:
            head = coco.loadAnns(headid)
            for h in head:
                tmp = {}
                tmp['local_path'] = image_data['local_path'].replace("sotra-small-pen_0", "small-pen-test-site_1")

                tmp['head_bbox'] = h['bbox']
                tmp['head_mask'] = h['segmentation']
                tmp['jfile'] = jfile

                eye_center_data.append(tmp)

In [ ]:
from tqdm import tqdm
import os
import skimage.io as io

In [ ]:
for ec in tqdm(eye_center_data):
    # load the image
    new_path = '/root/data/reidentification/heads/{}.head.jpg'.format(os.path.basename(ec['local_path']))
    if os.path.isfile(new_path):
        continue
    image = io.imread(ec['local_path'])
    
    x1, y1, width, height = [int(c) for c in ec['head_bbox']]
    head = image[y1:y1+height, x1:x1+width, :]
    
    io.imsave(new_path, head)

# Build generator

In [ ]:
import glob
import json
import os
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
import skimage.io as io
from skimage.transform import resize
from pycocotools.coco import COCO
import random
from keras.applications.mobilenet import MobileNet


### list images

In [ ]:
heads = glob.glob('/root/data/reidentification/heads_with_eye/small-pen-test-site/*')
print(len(heads))

In [ ]:
os.path.basename(heads[0]).split('.')[0]

In [ ]:
mappingdic = {}
all_path = glob.glob('/root/data/small_pen_data_collection/1*/*.jpg')
for path in all_path:
    mappingdic[os.path.basename(path).split('.')[0]] = path.split('/')[-2].split('_')[0]

In [ ]:
experiencedic = {}
for head in heads:
    k = os.path.basename(head).split('.')[0]
    exp = mappingdic[k]
    if exp not in experiencedic:
        experiencedic[exp] = []
    experiencedic[exp].append(head)
    

In [ ]:
for (k, v) in experiencedic.items():
    print(k, len(v))

In [ ]:
batch_size = 16
nclasses = len(experiencedic.keys())
print(nclasses)
input_size = (128, 128, 3)
experiences = list(experiencedic.keys())

In [ ]:
def generator(experiencedic, batch_size, input_size):
    
    while True:
        x_batch = np.zeros((batch_size, input_size[0], input_size[1], input_size[2]))
        y_batch = np.zeros((batch_size, nclasses))
        for i in range(batch_size):
            random_exp = np.random.choice(list(experiencedic.keys()))
            random_head = np.random.choice(experiencedic[random_exp])
            img = io.imread(random_head)
            head = resize(img, input_size)
            xscale = input_size[0] / img.shape[0]
            yscale = input_size[1] / img.shape[1]
            x_batch[i, ...] = head
            y_batch[i, experiences.index(random_exp)] = 1
        yield x_batch, y_batch

In [ ]:
train_gen = generator(experiencedic, batch_size, input_size)
# val_gen = generator(experiencedic, batch_size, input_size)

In [ ]:
xb, yb = next(train_gen)
for i in range(batch_size):
    f, ax = plt.subplots(1)
    ax.imshow(xb[i, ...])
    ax.set_title(experiences[np.argmax(yb[i, ...])])
    plt.show()

# LOAD MODEL

In [ ]:
import keras
from keras.applications.mobilenet import MobileNet
from keras import layers
from keras.models import Model
from keras.optimizers import Adam

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

In [ ]:
alpha = 1.0
dropout=1e-3
classes = nclasses
shape = (1, 1, int(1024 * alpha))

In [ ]:
mnet = MobileNet(input_shape=input_size, weights='imagenet', include_top=False)

In [ ]:
x = layers.GlobalAveragePooling2D()(mnet.output)
x = layers.Reshape(shape, name='reshape_1')(x)
x = layers.Dropout(dropout, name='dropout')(x)
x = layers.Conv2D(classes, (1, 1),
                  padding='same',
                  name='conv_preds')(x)
x = layers.Activation('softmax', name='act_softmax')(x)
x = layers.Reshape((classes,), name='reshape_2')(x)

In [ ]:
model = Model(inputs=mnet.inputs, outputs=[x])

In [ ]:
adam = Adam(lr=0.001)
model.compile(optimizer=adam, loss="categorical_crossentropy", metrics=["categorical_accuracy", "top_k_categorical_accuracy"])

In [ ]:
history = model.fit_generator(train_gen,
                    steps_per_epoch= len(heads) // batch_size, 
                    epochs=2)

In [ ]:
model.summary()

# CHECK WEIGHTS

In [ ]:
from __future__ import print_function

import numpy as np
import time
from keras.preprocessing.image import save_img
from keras.applications import vgg16
from keras import backend as K

In [ ]:
# dimensions of the generated pictures for each filter.
img_width = 128
img_height = 128

# the name of the layer we want to visualize
# (see model definition at keras/applications/vgg16.py)
layer_name = 'conv1'

# util function to convert a tensor into a valid image


def deprocess_image(x):
    # normalize tensor: center on 0., ensure std is 0.1
    x -= x.mean()
    x /= (x.std() + K.epsilon())
    x *= 0.1

    # clip to [0, 1]
    x += 0.5
    x = np.clip(x, 0, 1)

    # convert to RGB array
    x *= 255
    if K.image_data_format() == 'channels_first':
        x = x.transpose((1, 2, 0))
    x = np.clip(x, 0, 255).astype('uint8')
    return x


# # build the VGG16 network with ImageNet weights
# model = vgg16.VGG16(weights='imagenet', include_top=False)
# print('Model loaded.')

# model.summary()

# this is the placeholder for the input images
input_img = model.input

# get the symbolic outputs of each "key" layer (we gave them unique names).
layer_dict = dict([(layer.name, layer) for layer in model.layers[1:]])


def normalize(x):
    # utility function to normalize a tensor by its L2 norm
    return x / (K.sqrt(K.mean(K.square(x))) + K.epsilon())


kept_filters = []
for filter_index in range(32):
    # we only scan through the first 200 filters,
    # but there are actually 512 of them
    print('Processing filter %d' % filter_index)
    start_time = time.time()

    # we build a loss function that maximizes the activation
    # of the nth filter of the layer considered
    layer_output = layer_dict[layer_name].output
    if K.image_data_format() == 'channels_first':
        loss = K.mean(layer_output[:, filter_index, :, :])
    else:
        loss = K.mean(layer_output[:, :, :, filter_index])

    # we compute the gradient of the input picture wrt this loss
    grads = K.gradients(loss, input_img)[0]

    # normalization trick: we normalize the gradient
    grads = normalize(grads)

    # this function returns the loss and grads given the input picture
    iterate = K.function([input_img], [loss, grads])

    # step size for gradient ascent
    step = 1.

    # we start from a gray image with some random noise
    if K.image_data_format() == 'channels_first':
        input_img_data = np.random.random((1, 3, img_width, img_height))
    else:
        input_img_data = np.random.random((1, img_width, img_height, 3))
    input_img_data = (input_img_data - 0.5) * 20 + 128

    # we run gradient ascent for 20 steps
    for i in range(20):
        loss_value, grads_value = iterate([input_img_data])
        input_img_data += grads_value * step

        print('Current loss value:', loss_value)
        if loss_value <= 0.:
            # some filters get stuck to 0, we can skip them
            break

    # decode the resulting input image
    if loss_value > 0:
        img = deprocess_image(input_img_data[0])
        kept_filters.append((img, loss_value))
    end_time = time.time()
    print('Filter %d processed in %ds' % (filter_index, end_time - start_time))


In [ ]:

# we will stich the best 64 filters on a 8 x 8 grid.
n = 4

# the filters that have the highest loss are assumed to be better-looking.
# we will only keep the top 64 filters.
kept_filters.sort(key=lambda x: x[1], reverse=True)
kept_filters = kept_filters[:n * n]

# build a black picture with enough space for
# our 8 x 8 filters of size 128 x 128, with a 5px margin in between
margin = 5
width = n * img_width + (n - 1) * margin
height = n * img_height + (n - 1) * margin
stitched_filters = np.zeros((width, height, 3))

# fill the picture with our saved filters
for i in range(n):
    for j in range(n):
        img, loss = kept_filters[i * n + j]
        width_margin = (img_width + margin) * i
        height_margin = (img_height + margin) * j
        stitched_filters[
            width_margin: width_margin + img_width,
            height_margin: height_margin + img_height, :] = img

# save the result to disk
plt.imshow(stitched_filters)


In [ ]:
plt.imshow(kept_filters[10][0])